In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import os
import os
import shutil
import random

In [ ]:
SOURCE_DIR = "/PlantVillage"
TARGET_DIR = "/dataset"
TRAIN_SPLIT = 0.8
VALID_EXTENSIONS = [".jpg", ".jpeg", ".png"]

# Clear old dataset if exists
if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

for category in os.listdir(SOURCE_DIR):
    src_folder = os.path.join(SOURCE_DIR, category)
    if not os.path.isdir(src_folder):
        continue

    # Filter only valid image files
    images = [f for f in os.listdir(src_folder) if os.path.splitext(f)[1].lower() in VALID_EXTENSIONS]
    if not images:
        print(f"⚠️ No valid images found in: {category}")
        continue

    random.shuffle(images)

    train_count = int(len(images) * TRAIN_SPLIT)
    train_images = images[:train_count]
    val_images = images[train_count:]

    train_path = os.path.join(TARGET_DIR, "train", category)
    val_path = os.path.join(TARGET_DIR, "val", category)
    os.makedirs(train_path, exist_ok=True)
    os.makedirs(val_path, exist_ok=True)

    for img in train_images:
        shutil.copy(os.path.join(src_folder, img), os.path.join(train_path, img))
    for img in val_images:
        shutil.copy(os.path.join(src_folder, img), os.path.join(val_path, img))

    print(f"✅ {category}: {len(train_images)} train, {len(val_images)} val")

print("\n✅ Dataset split complete.")


In [3]:
from pathlib import Path

train_dir = Path("dataset/train")
val_dir = Path("dataset/val")

print("Train classes:", [x.name for x in train_dir.iterdir()])
print("Validation classes:", [x.name for x in val_dir.iterdir()])

Train classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage']
Validation classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage']


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    'dataset/train/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 2386 images belonging to 3 classes.


In [11]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=25)

Epoch 1/25
75/75 [==============================] - 65s 825ms/step - loss: 0.1882 - accuracy: 0.9191
Epoch 2/25
75/75 [==============================] - 61s 805ms/step - loss: 0.0698 - accuracy: 0.9774
Epoch 3/25
75/75 [==============================] - 55s 731ms/step - loss: 0.0588 - accuracy: 0.9811
Epoch 4/25
75/75 [==============================] - 55s 733ms/step - loss: 0.0455 - accuracy: 0.9807
Epoch 5/25
75/75 [==============================] - 51s 681ms/step - loss: 0.0381 - accuracy: 0.9853
Epoch 6/25
75/75 [==============================] - 56s 737ms/step - loss: 0.0321 - accuracy: 0.9866
Epoch 7/25
75/75 [==============================] - 56s 741ms/step - loss: 0.0333 - accuracy: 0.9870
Epoch 8/25
75/75 [==============================] - 61s 809ms/step - loss: 0.0443 - accuracy: 0.9837
Epoch 9/25
75/75 [==============================] - 49s 652ms/step - loss: 0.0410 - accuracy: 0.9841
Epoch 10/25
75/75 [==============================] - 49s 659ms/step - loss: 0.0296 - accura

In [1]:
model.save("plant_disease_model.keras")

NameError: name 'model' is not defined